In [1]:
# 全局设置
import os
import datetime as dt
import time

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
Factorize = QS.FactorDB.Factorize

In [3]:
# 示例因子库
# FDB = QS.FactorDB.HDF5DB(sys_args={"主目录": "../Data/HDF5"}).connect();
FDB = QS.FactorDB.HDF5DB(config_file="../config/HDF5DBConfig.json").connect();

In [4]:
# 示例因子表
CFT = QS.FactorDB.CustomFT(name="示例因子表")

FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")
Mid = Factorize((High + Low) / 2, factor_name="mid")

CFT.addFactors(factor_list=[Mid, High, Low])
CFT.addFactors(factor_table=FT, factor_names=["open", "close"])

DTs = FT.getDateTime()
CFT.setDateTime(dts=DTs)

IDs = FT.getID()
CFT.setID(ids=IDs)

print("因子 : ", CFT.FactorNames)
print(f"时点 : {DTs[0]} ~ {DTs[-1]}, 共 {len(DTs)} 个")
print(f"ID : {IDs[0]} ~ {IDs[-1]}, 共 {len(IDs)} 个")

因子 :  ['close', 'high', 'low', 'mid', 'open']
时点 : 2000-01-04 00:00:00 ~ 2019-04-30 00:00:00, 共 4681 个
ID : 000001.SZ ~ T00018.SH, 共 3710 个


# 随机访问模式

参见: [数据读取](./数据读写.ipynb#读取数据)

串行递归方式计算因子图，返回因子数据。

In [5]:
# 随机访问模式
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,15), end_dt=dt.datetime(2018,11,22))
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]

Data = CFT.readData(factor_names=["close", "mid"], ids=IDs, dts=DTs)
print(Data)

<class 'QuantStudio.Tools.QSObjects.Panel'>
Dimensions: 2 (items) x 6 (major_axis) x 3 (minor_axis)
Items axis: close to mid
Major_axis axis: 2018-11-15 00:00:00 to 2018-11-22 00:00:00
Minor_axis axis: 000001.SZ to 000003.SZ


# 时序遍历模式

![时序遍历模式](../images/遍历模式.png)

遍历模式的相关参数集:
* 缓冲模式: str, 可选 "因子"(默认值), "ID"
* 最大缓冲因子数: int, 默认值 60, 当缓冲模式为 "因子" 时生效
* 最大缓冲ID数: int, 默认值 10000, 当缓冲模式为 "ID" 时生效
* 向前缓冲时点数: int, 默认值 600,
* 向后缓冲时点数: int, 默认值 1,
* 缓冲区大小: int, 默认值 300, 单位 MB
* 遍历时点: list(datetime.datetime), 默认值 []
* 遍历ID: list(str), 默认值 []

start(dts, ids=None, **kwargs)

启动遍历模式:	
* dts: list(datetime.datetime), 待遍历的时点序列
* ids: list(str), 遍历过程中会使用到的 ID 序列, 如果为 None 表示因子表的 ID 序列
* 返回: int, ErrorCode

move(idt, **kwargs)

遍历运行至某个时点时调用:
* idt: datetime.datetime, 遍历中的某个时点
* 返回: int, ErrorCode

end()

结束遍历模式
* 返回: int, ErrorCode

In [6]:
# 时序遍历模式
DTs = CFT.getDateTime()
IDs = CFT.getID()

CFT.Args["遍历模式"]["向前缓冲时点数"] = 60
CFT.Args["遍历模式"]["向后缓冲时点数"] = 0

TestDTs = DTs[:480]

StartT = time.perf_counter()
for i, iDT in enumerate(TestDTs):
    iData = CFT.readData(factor_names=CFT.FactorNames, ids=IDs, dts=[iDT])
print("随机访问模式运行时间 : ", time.perf_counter() - StartT)

StartT = time.perf_counter()

with CFT.start(dts=TestDTs) as CFT:
    for i, iDT in enumerate(TestDTs):
        iData = CFT.readData(factor_names=CFT.FactorNames, ids=IDs, dts=[iDT])

# CFT.start(dts=TestDTs)
# for i, iDT in enumerate(TestDTs):
#     CFT.move(iDT)
#     iData = CFT.readData(factor_names=CFT.FactorNames, ids=IDs, dts=[iDT])
# CFT.end()

print("时序遍历模式运行时间 : ", time.perf_counter() - StartT)

时序遍历模式运行时间 :  42.12831925000137


# 批量运算模式

write2FDB(factor_names, ids, dts, factor_db, table_name, if_exists="update", subprocess_num=cpu_count()-1, dt_ruler=None, section_ids=None, specific_target={}, **kwargs)

将因子表中因子数据写入因子库中:	
* factor_names: list(str), 因子名列表
* ids: list(str), ID 序列
* dts: list(datetime.datetime), 时点序列
* factor_db: WritableFactorDB, 支持写入操作的因子库对象
* table_name: str, 数据写入的表名
* if_exists: str, 默认值 "update", 参见 [因子库对象 writeData 方法](./数据读写.ipynb#数据写入)
* subprocess_num: int, 子进程数, 数据计算和写入时开启的最大子进程个数
* dt_ruler: list(datetime.datetime), 时间标尺, 在进行回溯计算时所依赖的时点序列, 如果为 None 则以 dts 作为时间标尺
* section_ids: list(str) 或者 None(默认值), 截面 ID 序列, 如果为 None 表示其等于参数 ids
* specific_target: dict, 默认值 {}, 特别指定的目标写入位置
* kwargs:
    * cache_dir: 计算过程中缓存文件存放的目录, 默认调用 tempfile.TemporaryDirectory() 创建临时目录

TODO:
* 仿照时序遍历模式实现, 进入模式后 readData 起效

In [7]:
# 批量运算模式

SectionIDs = None
DTRuler = CFT.getDateTime()
TargetTable = "test_table"

CFT.write2FDB(factor_names=CFT.FactorNames, 
              ids=CFT.getID(), 
              dts=DTRuler[20:], 
              factor_db=FDB, 
              table_name=TargetTable, 
              if_exists="update", 
              subprocess_num=3, 
              dt_ruler=DTRuler, 
              section_ids=SectionIDs,
              specific_target={})

print(FDB.TableNames)
print(FDB.getTable(TargetTable).FactorNames)

==========因子运算==========
1. 原始数据准备



100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


耗时 : 8.66
2. 因子数据计算



100% (15 of 15) |########################| Elapsed Time: 0:00:13 Time:  0:00:13


耗时 : 37.83
3. 清理缓存

耗时 : 0.30
总耗时 : 46.79
['index_cn_day_bar', 'stock_cn_day_bar_adj_backward_nafilled', 'stock_cn_day_bar_nafilled', 'stock_cn_factor_barra', 'stock_cn_factor_barra_descriptor', 'stock_cn_factor_momentum', 'stock_cn_factor_technical', 'stock_cn_index_component', 'stock_cn_industry', 'stock_cn_info', 'stock_cn_multi_factor_classic', 'test_table']
['close', 'high', 'low', 'mid', 'open']


In [8]:
# 清除数据
FDB.deleteTable(TargetTable);